In [ ]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 743 kB 4.2 MB/s 


In [ ]:
import pickle
filename = '/content/drive/MyDrive/CRF_model/model6.sav'
crf_model=pickle.load(open(filename,'rb'))

In [ ]:
!pip install flair

     |████████████████████████████████| 322 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 45.1 MB/s 
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 61 kB 441 kB/s 
     |████████████████████████████████| 788 kB 55.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 4.7 MB/s 
     |████████████████████████████████| 3.4 MB 55.1 MB/s 
     |████████████████████████████████| 981 kB 52.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 596 kB 66.4 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=5eaf6cfdcbca4d02d06ca157efff0620d268891cb05cff

In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
tagger = SequenceTagger.load('pos')
words = []
POS = []
sentence = Sentence("Arjuna and Draupadi lived in Hastinapura")
tagger.predict(sentence)
pos_tags = sentence.to_tagged_string()
print(pos_tags)
pos_tags_split = pos_tags.split()
print(pos_tags_split)
for i in range(0, len(pos_tags_split), 2):
  words.append([pos_tags_split[i],pos_tags_split[i+1]])

print(words)

2022-01-11 08:18:58,763 --------------------------------------------------------------------------------
2022-01-11 08:18:58,764 The model key 'pos' now maps to 'https://huggingface.co/flair/pos-english' on the HuggingFace ModelHub
2022-01-11 08:18:58,771  - The most current version of the model is automatically downloaded from there.
2022-01-11 08:18:58,778  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt)
2022-01-11 08:18:58,779 --------------------------------------------------------------------------------


Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

2022-01-11 08:19:06,556 loading file /root/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63
Arjuna <NNP> and <CC> Draupadi <NNP> lived <VBD> in <IN> Hastinapura <NNP>
['Arjuna', '<NNP>', 'and', '<CC>', 'Draupadi', '<NNP>', 'lived', '<VBD>', 'in', '<IN>', 'Hastinapura', '<NNP>']
[['Arjuna', '<NNP>'], ['and', '<CC>'], ['Draupadi', '<NNP>'], ['lived', '<VBD>'], ['in', '<IN>'], ['Hastinapura', '<NNP>']]


In [ ]:
s = words
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    # other than first word 
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
ip = sent2features(s)

In [ ]:
print([ip])

[[{'bias': 1.0, 'word.lower()': 'arjuna', 'word[-3:]': 'una', 'word[-2:]': 'na', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': '<NNP>', 'postag[:2]': '<N', 'BOS': True, '+1:word.lower()': 'and', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': '<CC>', '+1:postag[:2]': '<C'}, {'bias': 1.0, 'word.lower()': 'and', 'word[-3:]': 'and', 'word[-2:]': 'nd', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': '<CC>', 'postag[:2]': '<C', '-1:word.lower()': 'arjuna', '-1:word.istitle()': True, '-1:word.isupper()': False, '-1:postag': '<NNP>', '-1:postag[:2]': '<N', '+1:word.lower()': 'draupadi', '+1:word.istitle()': True, '+1:word.isupper()': False, '+1:postag': '<NNP>', '+1:postag[:2]': '<N'}, {'bias': 1.0, 'word.lower()': 'draupadi', 'word[-3:]': 'adi', 'word[-2:]': 'di', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': '<NNP>', 'postag[:2]': '<N', '-1:word.lower()': 'an

In [ ]:
pred = crf_model.predict([ip])

In [ ]:
print(pred)

[['B-PER', 'O', 'B-PER', 'O', 'O', 'B-PLACE']]
